In [9]:
import json
import os
import pandas as pd

jsonfile = './dune_1500k.json'
json_file = json.load(open(jsonfile))
df = pd.DataFrame(columns=["rk","ua","rs","tc","amt","cc","dwm","lzd","ibt"]) 
# 将数据转换为 DataFrame 并进行合并
data_list = []
for data in json_file['data']['get_execution']['execution_succeeded']['data']:
    data_list.append(data)

# 使用 pandas.concat 进行合并
df = pd.concat([df, pd.DataFrame(data_list)], ignore_index=True)

# 将 'rk' 列设置为索引，并删除默认索引
df.set_index('rk', inplace=True)
print(len(df))
print(df)

csvfile = './initialList.csv'
address_df = pd.read_csv(csvfile)
print('address_df',address_df)

address_list = address_df['ADDRESS'].tolist()
print('sybil list:',len(address_list))

df = df[~df['ua'].isin(address_list)]

print(len(df))
print(df)


1495498
                                                 ua  rs     tc          amt  \
rk                                                                            
1        0x65a8f07bd9a8598e1b5b6c0a88f4779dbc077675  43  12117  12694101.42   
2        0x4398805dbcd8c08e45fa87070bedb69ab238ddd3  43   2450   3374087.10   
3        0x19a2fcfa3916a4b77a0c14e3a28a24f43081a341  43   1760   1239422.53   
4        0x172f86ac7711f5dc024934649353bf864af5055c  43    793   1029492.25   
5        0x4cb6f0ef0eeb503f8065af1a6e6d5dd46197d3d9  42  39187  10971047.50   
...                                             ...  ..    ...          ...   
1495494  0x53cafbbce66f51db79fd98ca3bc8324b8f910a3b  17     25     23217.38   
1495495  0x54d82287a9339daaf5b0a5fba160845872fa0409  17     25     23217.34   
1495496  0x338a64d348f034c3529d846c99609c6331c36763  17     55     23216.40   
1495497  0xb914cb625b51e9b8f78384189e6fa77eb515fc8e  17     10     23215.77   
1495498  0xba834014c0e9625f5fcb488a5e37aa473

In [10]:
# 分组并计算平均值和记录数
grouped = df.groupby(['cc', 'dwm', 'lzd'])
grouped_stats = grouped['amt'].agg(['mean', 'count']).reset_index()
grouped_stats.columns = ['cc', 'dwm', 'lzd', 'mean_amt', 'count']

# 过滤记录数大于 20 的分组，且跨链金额大于100
filtered_groups = grouped_stats[(grouped_stats['count'] > 20) & (grouped_stats['mean_amt'] > 100)]


# 打印分组的 ID 及其记录数，从 1 开始编号
for idx, row in enumerate(filtered_groups.itertuples(), start=1):
    print(f"Group ID: [{idx}](cc={row.cc}, dwm={row.dwm}, lzd={row.lzd}), Record Count: {row.count}")


Group ID: [1](cc=10 / 14 / 19, dwm=17 / 12 / 8, lzd=368), Record Count: 21
Group ID: [2](cc=2 / 5 / 5, dwm=12 / 11 / 9, lzd=501), Record Count: 23
Group ID: [3](cc=3 / 6 / 4, dwm=16 / 13 / 14, lzd=415), Record Count: 41
Group ID: [4](cc=3 / 8 / 6, dwm=34 / 17 / 11, lzd=405), Record Count: 32
Group ID: [5](cc=4 / 14 / 4, dwm=15 / 10 / 6, lzd=294), Record Count: 21
Group ID: [6](cc=4 / 5 / 10, dwm=9 / 9 / 9, lzd=546), Record Count: 24
Group ID: [7](cc=4 / 5 / 11, dwm=10 / 10 / 10, lzd=548), Record Count: 27
Group ID: [8](cc=4 / 5 / 11, dwm=11 / 10 / 10, lzd=550), Record Count: 25
Group ID: [9](cc=4 / 5 / 5, dwm=10 / 10 / 7, lzd=528), Record Count: 32
Group ID: [10](cc=4 / 6 / 9, dwm=16 / 13 / 9, lzd=416), Record Count: 37
Group ID: [11](cc=4 / 6 / 9, dwm=16 / 14 / 9, lzd=416), Record Count: 65
Group ID: [12](cc=4 / 8 / 10, dwm=16 / 12 / 6, lzd=426), Record Count: 21
Group ID: [13](cc=4 / 8 / 7, dwm=30 / 16 / 9, lzd=405), Record Count: 21
Group ID: [14](cc=5 / 10 / 10, dwm=15 / 10 / 7, lz

In [14]:
# 保存 'rk' 列并重置索引
df['rk'] = df.index
df = df.reset_index(drop=True)

# 合并平均值到原始 DataFrame
df = df.merge(grouped_stats, on=['cc', 'dwm', 'lzd'], how='left')
print("Columns after merge:", df.columns)
# 删除多余的列，并将 count_x 重命名为 count
df['count'] = df['count_x']
df['mean_amt'] = df['mean_amt_x']
df = df.drop(columns=['count_x', 'count_y','mean_amt_x','mean_amt_y'])

# 调试步骤：打印合并后的 DataFrame 列名，检查 'mean_amt' 是否存在
print("Columns after merge:", df.columns)

# 过滤出 amt 在平均值正负10%范围内的记录
df_filtered = df[(df['amt'] >= df['mean_amt'] * 0.9) & (df['amt'] <= df['mean_amt'] * 1.1)]

# 再次分组并计算记录数
filtered_grouped = df_filtered.groupby(['cc', 'dwm', 'lzd']).filter(lambda x: len(x) > 20)

# 增加 groupid 列
filtered_grouped['groupid'] = filtered_grouped.groupby(['cc', 'dwm', 'lzd']).ngroup() + 1

# 按照 groupid 和 rk 排序
filtered_grouped = filtered_grouped.sort_values(by=['groupid', 'rk'])

# 将 'rk' 列重新设置为索引
filtered_grouped.set_index('rk', inplace=True)

for idx, row in enumerate(filtered_groups.itertuples(), start=1):
    print(f"Group ID: [{idx}](cc={row.cc}, dwm={row.dwm}, lzd={row.lzd}), Record Count: {row.count}")

print(filtered_grouped)

Columns after merge: Index(['index', 'ua', 'rs', 'tc', 'amt', 'cc', 'dwm', 'lzd', 'ibt', 'count_x',
       'mean_amt_x', 'rk', 'mean_amt_y', 'count_y'],
      dtype='object')
Columns after merge: Index(['index', 'ua', 'rs', 'tc', 'amt', 'cc', 'dwm', 'lzd', 'ibt', 'rk',
       'count', 'mean_amt'],
      dtype='object')
Group ID: [1](cc=10 / 14 / 19, dwm=17 / 12 / 8, lzd=368), Record Count: 21
Group ID: [2](cc=2 / 5 / 5, dwm=12 / 11 / 9, lzd=501), Record Count: 23
Group ID: [3](cc=3 / 6 / 4, dwm=16 / 13 / 14, lzd=415), Record Count: 41
Group ID: [4](cc=3 / 8 / 6, dwm=34 / 17 / 11, lzd=405), Record Count: 32
Group ID: [5](cc=4 / 14 / 4, dwm=15 / 10 / 6, lzd=294), Record Count: 21
Group ID: [6](cc=4 / 5 / 10, dwm=9 / 9 / 9, lzd=546), Record Count: 24
Group ID: [7](cc=4 / 5 / 11, dwm=10 / 10 / 10, lzd=548), Record Count: 27
Group ID: [8](cc=4 / 5 / 11, dwm=11 / 10 / 10, lzd=550), Record Count: 25
Group ID: [9](cc=4 / 5 / 5, dwm=10 / 10 / 7, lzd=528), Record Count: 32
Group ID: [10](cc=4 / 

In [15]:
# 保存 DataFrame 为 CSV 文件
csv_file_path = './dune_1500k_group_v2.csv'
filtered_grouped.to_csv(csv_file_path)